<a href="https://colab.research.google.com/github/kavyajeetbora/modern_geospatial_stack/blob/master/notebooks/DuckDB_in_Jupyter_Notebooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DuckDB in Jupyter Notebooks
A streamlined workflow for SQL analysis with DuckDB and Jupyter

## Library Import and Configuration

In [1]:
!pip install --quiet duckdb
!pip install --quiet jupysql
!pip install --quiet duckdb-engine
!pip install --quiet pandas
!pip install --quiet matplotlib
!pip install -q osmnx
!pip install -q pydeck

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.9/95.9 kB 971.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.8/249.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 826.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.9 MB/s eta 0:00:00


In [2]:
import duckdb
import pandas as pd
import geopandas as gpd
import shapely
import osmnx as ox
import pydeck as pdk
# No need to import sqlalchemy or duckdb_engine
#  JupySQL will use SQLAlchemy to auto-detect the driver needed based on your connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

Deploy Dash apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


We configure jupysql to return data as a Pandas dataframe and have less verbose output

In [3]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

## Connecting to DuckDB
Connect jupysql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.

In [7]:
%sql duckdb:///:memory:
# %sql duckdb:///path/to/file.db


In [8]:
%%sql
INSTALL httpfs;
INSTALL spatial;

,Success


## Define the area of Interest

In [ ]:
# W,S,E,N =  72.824548,19.19574,72.869386,19.231531

W,S,E,N = 72.8457167244,19.1174666804,72.8505208832,19.1231406236

## Downloading Station Nodes from OSM

In [9]:
public_transport = ox.features.features_from_bbox(bbox=(N,S,E,W), tags={'public_transport':'station'}).reset_index()
pb = public_transport[public_transport['element_type']=='node'].copy()
pb_gdf = pb[['public_transport', 'geometry']]
pb_gdf.sample(min(5,len(pb_gdf)))

,public_transport,geometry
0,station,POINT (72.84642 19.11970)
1,station,POINT (72.84879 19.12046)


In [11]:
pb_gdf.to_parquet('stations.parquet')

## Downloading the Buildings data from overture maps

In [13]:
df = duckdb.read_parquet('''s3://overturemaps-us-west-2/release/2024-06-13-beta.0/theme=transportation/type=*/*''')
df

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬─────────┐
│          id          │       geometry       │         bbox         │ … │     width_rules      │  road   │
│       varchar        │         blob         │ struct(xmin float,…  │   │ struct("value" dou…  │ varchar │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼─────────┤
│ 08ff39d86161931104…  │ \x00\x00\x00\x00\x…  │ {'xmin': -180.0, '…  │ … │ NULL                 │ NULL    │
│ 08ff2a06d8692b6004…  │ \x00\x00\x00\x00\x…  │ {'xmin': -157.3309…  │ … │ NULL                 │ NULL    │
│ 08fd46b2c6d9cdb204…  │ \x00\x00\x00\x00\x…  │ {'xmin': -176.2032…  │ … │ NULL                 │ NULL    │
│ 08fbb364e1ada63604…  │ \x00\x00\x00\x00\x…  │ {'xmin': -176.6684…  │ … │ NULL                 │ NULL    │
│ 08fbb364e1adecdd04…  │ \x00\x00\x00\x00\x…  │ {'xmin': -176.6684…  │ … │ NULL                 │ NULL    │
│ 08fbb364e132c2ab04…  │ \x0

BBOX for Mumbai:

```python
'Mumbai': (72.7712628039,18.8895552464,73.0912396106,19.322318006),
```

In [35]:
%%time

buildings_data_url = r"'s3://overturemaps-us-west-2/release/2024-06-13-beta.0/theme=buildings/type=*/*'"
stations_data_url = "'stations.parquet'"

df = duckdb.sql(
    f'''
    LOAD spatial;
    LOAD httpfs;
    COPY(
        SELECT
            id,
            names.primary as name,
            height,
            ST_GeomFromWKB(buildings.geometry) as geom
        FROM read_parquet({buildings_data_url}, filename=true, hive_partitioning=1) AS buildings
        JOIN read_parquet({stations_data_url}) as stations
        ON ST_Intersects(ST_GeomFromWKB(buildings.geometry), ST_GeomFromWKB(stations.geometry))
        WHERE buildings.bbox.xmin > {W}
        AND buildings.bbox.xmax < {E}
        AND buildings.bbox.ymin > {S}
        AND buildings.bbox.ymax < {N}
    ) TO 'station_buildings.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON');
    '''
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 6.01 s, sys: 512 ms, total: 6.52 s
Wall time: 2min 9s


In [41]:
gdf = gpd.read_file('station_buildings.geojson')
gdf

,id,names,height,geometry
0,08b608b096140fff0200332fd99e2ab0,None,None,"POLYGON ((72.84658 19.12116, 72.84633 19.11941..."
1,08b608b09610cfff0200839516a02212,Andheri,None,"POLYGON ((72.84799 19.12082, 72.84819 19.12070..."


## Intersection of builings and stations using Geopandas

[Spatial joins using geopandas](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html)

[Predicates of spatial joins in geopanda](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html#binary-predicate-joins)

## Plot the results on a map

In [53]:
def create_map(W,S,E,N, geojson_file, point_layer):
    bbox_geom = shapely.geometry.box(W,S,E,N)
    boundary_json = eval(gpd.GeoSeries(bbox_geom).to_json())

    boundary_layer = pdk.Layer(
        "GeoJsonLayer",
        boundary_json,
        opacity=1,
        stroked=True,
        filled=False,
        get_line_color=[255, 255, 0]
    )

    layer = pdk.Layer(
        "GeoJsonLayer",
        geojson_file,
        opacity=1,
        stroked=True,
        filled=True,
        get_fill_color=[100, 200, 0],
        get_line_color=[0,100,0],
        pickable=True
    )


    ## Add point layers
    nodes = pdk.Layer(
        "GeoJsonLayer",
        point_layer,
        opacity=1,
        stroked=True,
        filled=True,
        get_fill_color=[200, 100, 156],
        get_line_color=[200, 100, 156],
        pickable=True
    )

    layers = [boundary_layer, layer, nodes]

    C = bbox_geom.centroid
    view_state = pdk.ViewState(latitude=C.y, longitude=C.x, zoom=15, bearing=0, pitch=45)
    # Render

    r = pdk.Deck(layers=layers, initial_view_state=view_state, tooltip = True)
    return r

In [49]:
%%time
buildings_gdf = gpd.read_file("station_buildings.geojson").fillna("Na")
print(buildings_gdf.shape[0])
#buildings_gdf = buildings_gdf.sample(1000)
geojson = eval(buildings_gdf.to_json())

pt_geojson = eval(pb_gdf.to_json())

2
CPU times: user 31.3 ms, sys: 978 µs, total: 32.2 ms
Wall time: 32.9 ms


In [54]:
Map = create_map(W,S,E,N, geojson_file=geojson, point_layer=pt_geojson)
Map

<IPython.core.display.Javascript object>

{
  "initialViewState": {
    "bearing": 0,
    "latitude": 19.120303652,
    "longitude": 72.84811880379999,
    "pitch": 45,
    "zoom": 15
  },
  "layers": [
    {
      "@@type": "GeoJsonLayer",
      "data": {
        "bbox": [
          72.8457167244,
          19.1174666804,
          72.8505208832,
          19.1231406236
        ],
        "features": [
          {
            "bbox": [
              72.8457167244,
              19.1174666804,
              72.8505208832,
              19.1231406236
            ],
            "geometry": {
              "coordinates": [
                [
                  [
                    72.8505208832,
                    19.1174666804
                  ],
                  [
                    72.8505208832,
                    19.1231406236
                  ],
                  [
                    72.8457167244,
                    19.1231406236
                  ],
                  [
                    72.8457167244,
                    19.1174666804
                  ],
                  [
                    72.8505208832,
                    19.1174666804
                  ]
                ]
              ],
              "type": "Polygon"
            },
            "id": "0",
            "properties": {},
            "type": "Feature"
          }
        ],
        "type": "FeatureCollection"
      },
      "filled": false,
      "getLineColor": [
        255,
        255,
        0
      ],
      "id": "7efd0296-31cd-4724-ad9a-dd435915024f",
      "opacity": 1,
      "stroked": true
    },
    {
      "@@type": "GeoJsonLayer",
      "data": {
        "features": [
          {
            "geometry": {
              "coordinates": [
                [
                  [
                    72.84658,
                    19.1211582
                  ],
                  [
                    72.8463332,
                    19.1194121
                  ],
                  [
                    72.8464244,
                    19.1193335
                  ],
                  [
                    72.8463171,
                    19.1184693
                  ],
                  [
                    72.8464503,
                    19.1184545
                  ],
                  [
                    72.8467658,
                    19.1207171
                  ],
                  [
                    72.8468134,
                    19.1211277
                  ],
                  [
                    72.84658,
                    19.1211582
                  ]
                ]
              ],
              "type": "Polygon"
            },
            "id": "0",
            "properties": {
              "height": "Na",
              "id": "08b608b096140fff0200332fd99e2ab0",
              "names": "Na"
            },
            "type": "Feature"
          },
          {
            "geometry": {
              "coordinates": [
                [
                  [
                    72.8479935,
                    19.1208236
                  ],
                  [
                    72.8481877,
                    19.1207004
                  ],
                  [
                    72.84945,
                    19.1198999
                  ],
                  [
                    72.8495577,
                    19.1200515
                  ],
                  [
                    72.8481013,
                    19.1209753
                  ],
                  [
                    72.8479935,
                    19.1208236
                  ]
                ]
              ],
              "type": "Polygon"
            },
            "id": "1",
            "properties": {
              "height": "Na",
              "id": "08b608b09610cfff0200839516a02212",
              "names": "Andheri"
            },
            "type": "Feature"
          }
        ],
        "type": "FeatureCollection"
      },
      "filled": true,
      "getFillColor": [
       